# Process the Summary Files

This script will process the *_summary.csv files.  The will iterate through all of the files and generate a single file for a specific growth phase and year.

In [4]:
#Import the required libraries
import sys
import os
import datetime
import numpy as np
import pandas as pd

In [7]:
sourcedir = "/OSM/CBR/AG_WHEATTEMP/work/output"

In [8]:
filelist_df = pd.DataFrame(columns=['filename'])
filelist_df.filename = sorted(sourcedir+'/'+f for f in os.listdir(sourcedir) if f.endswith('_summary.csv'))
print(filelist_df.head())


                                            filename
0  /OSM/CBR/AG_WHEATTEMP/work/output/113.60_-28.3...
1  /OSM/CBR/AG_WHEATTEMP/work/output/113.70_-28.4...
2  /OSM/CBR/AG_WHEATTEMP/work/output/113.70_-28.5...
3  /OSM/CBR/AG_WHEATTEMP/work/output/113.75_-28.4...
4  /OSM/CBR/AG_WHEATTEMP/work/output/113.75_-28.5...


In [55]:
count = 0
for fname in filelist_df.filename:
    count = count + 1
    print(fname)
    
    if count > 100:
        break


/OSM/CBR/AG_WHEATTEMP/work/output/113.60_-28.30_summary.csv
/OSM/CBR/AG_WHEATTEMP/work/output/113.70_-28.45_summary.csv
/OSM/CBR/AG_WHEATTEMP/work/output/113.70_-28.50_summary.csv
/OSM/CBR/AG_WHEATTEMP/work/output/113.75_-28.45_summary.csv
/OSM/CBR/AG_WHEATTEMP/work/output/113.75_-28.50_summary.csv
/OSM/CBR/AG_WHEATTEMP/work/output/113.80_-28.45_summary.csv
/OSM/CBR/AG_WHEATTEMP/work/output/113.80_-28.70_summary.csv
/OSM/CBR/AG_WHEATTEMP/work/output/113.80_-28.75_summary.csv
/OSM/CBR/AG_WHEATTEMP/work/output/113.80_-28.80_summary.csv
/OSM/CBR/AG_WHEATTEMP/work/output/113.85_-28.70_summary.csv
/OSM/CBR/AG_WHEATTEMP/work/output/113.85_-28.75_summary.csv
/OSM/CBR/AG_WHEATTEMP/work/output/113.85_-28.90_summary.csv
/OSM/CBR/AG_WHEATTEMP/work/output/113.90_-28.65_summary.csv
/OSM/CBR/AG_WHEATTEMP/work/output/113.90_-28.90_summary.csv
/OSM/CBR/AG_WHEATTEMP/work/output/113.95_-27.25_summary.csv
/OSM/CBR/AG_WHEATTEMP/work/output/113.95_-28.85_summary.csv
/OSM/CBR/AG_WHEATTEMP/work/output/113.95

In [34]:
#set some defaults to work with
filter_phase = "07_GrainFilling"
filter_year = 1957

In [42]:
dbname = filelist_df.filename[2]
dbname

'/OSM/CBR/AG_WHEATTEMP/work/output/113.70_-28.50_summary.csv'

In [43]:
dfData = pd.read_csv(dbname)
dfData

,SimID,variety,long,lat,sowingdate,phases,season_cumAvgTemp,season_cumApsimTT,season_cumRain,startDate,...,days<=0,days<=-1,days<=-2,days<=-3,days>=30,days>=32,days>=34,days>=36,days>=38,days>=40
0,496,mace,113.7,-28.5,1957-05-27,01_Germinating,37.60,34.725924,7.6,1957-05-27,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,496,mace,113.7,-28.5,1957-05-27,02_Emerging,90.45,85.534483,9.5,1957-05-29,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,496,mace,113.7,-28.5,1957-05-27,03_Vegetative,264.95,247.992277,135.8,1957-06-01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,496,mace,113.7,-28.5,1957-05-27,04_StemElongation,1057.95,1027.045018,312.6,1957-06-10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,496,mace,113.7,-28.5,1957-05-27,05_EarlyReproductive,1405.30,1369.560920,395.0,1957-08-03,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,496,mace,113.7,-28.5,1957-05-27,06_GrainSet,1597.95,1561.562358,403.9,1957-08-26,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,496,mace,113.7,-28.5,1957-05-27,07_GrainFilling,2318.05,2256.071910,409.9,1957-09-08,...,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN
7,496,mace,113.7,-28.5,1957-05-27,08_Maturing,2346.35,2284.774561,413.5,1957-10-20,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,496,mace,113.7,-28.5,1957-05-27,09_Ripening,2699.35,2580.627539,420.2,1957-10-22,...,NaN,NaN,NaN,NaN,5.0,4.0,1.0,NaN,NaN,NaN
9,496,mace,113.7,-28.5,1958-05-27,01_Germinating,32.60,33.298029,1.2,1958-05-27,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [44]:
dfData = dfData[(dfData['phases'] == filter_phase)]
dfData

,SimID,variety,long,lat,sowingdate,phases,season_cumAvgTemp,season_cumApsimTT,season_cumRain,startDate,...,days<=0,days<=-1,days<=-2,days<=-3,days>=30,days>=32,days>=34,days>=36,days>=38,days>=40
6,496,mace,113.7,-28.5,1957-05-27,07_GrainFilling,2318.05,2256.071910,409.9,1957-09-08,...,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN
15,496,mace,113.7,-28.5,1958-05-27,07_GrainFilling,2356.65,2279.326048,474.0,1958-09-04,...,NaN,NaN,NaN,NaN,2.0,2.0,NaN,NaN,NaN,NaN
25,496,mace,113.7,-28.5,1959-05-27,07_GrainFilling,2208.85,2128.060868,362.2,1959-08-23,...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
35,496,mace,113.7,-28.5,1960-05-27,07_GrainFilling,2318.15,2247.336306,356.3,1960-09-09,...,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN
44,496,mace,113.7,-28.5,1961-05-27,07_GrainFilling,2268.80,2170.998617,334.5,1961-09-04,...,NaN,NaN,NaN,NaN,5.0,2.0,1.0,NaN,NaN,NaN
53,496,mace,113.7,-28.5,1962-05-27,07_GrainFilling,2331.70,2262.822160,363.2,1962-08-30,...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
63,496,mace,113.7,-28.5,1963-05-27,07_GrainFilling,2252.45,2108.388637,457.1,1963-08-31,...,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN
73,496,mace,113.7,-28.5,1964-05-27,07_GrainFilling,2339.70,2237.335945,516.3,1964-08-26,...,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN
83,496,mace,113.7,-28.5,1965-05-27,07_GrainFilling,2332.45,2233.923648,542.7,1965-09-01,...,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN
93,496,mace,113.7,-28.5,1966-05-27,07_GrainFilling,2301.45,2244.784630,311.5,1966-09-06,...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN


In [45]:
dfData['sowingdate'] = pd.to_datetime(dfData['sowingdate'], format="%Y-%m-%d")
dfData

/apps/python/3.6.1/lib/python3.6/site-packages/IPython/kernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


,SimID,variety,long,lat,sowingdate,phases,season_cumAvgTemp,season_cumApsimTT,season_cumRain,startDate,...,days<=0,days<=-1,days<=-2,days<=-3,days>=30,days>=32,days>=34,days>=36,days>=38,days>=40
6,496,mace,113.7,-28.5,1957-05-27,07_GrainFilling,2318.05,2256.071910,409.9,1957-09-08,...,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN
15,496,mace,113.7,-28.5,1958-05-27,07_GrainFilling,2356.65,2279.326048,474.0,1958-09-04,...,NaN,NaN,NaN,NaN,2.0,2.0,NaN,NaN,NaN,NaN
25,496,mace,113.7,-28.5,1959-05-27,07_GrainFilling,2208.85,2128.060868,362.2,1959-08-23,...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
35,496,mace,113.7,-28.5,1960-05-27,07_GrainFilling,2318.15,2247.336306,356.3,1960-09-09,...,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN
44,496,mace,113.7,-28.5,1961-05-27,07_GrainFilling,2268.80,2170.998617,334.5,1961-09-04,...,NaN,NaN,NaN,NaN,5.0,2.0,1.0,NaN,NaN,NaN
53,496,mace,113.7,-28.5,1962-05-27,07_GrainFilling,2331.70,2262.822160,363.2,1962-08-30,...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
63,496,mace,113.7,-28.5,1963-05-27,07_GrainFilling,2252.45,2108.388637,457.1,1963-08-31,...,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN
73,496,mace,113.7,-28.5,1964-05-27,07_GrainFilling,2339.70,2237.335945,516.3,1964-08-26,...,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN
83,496,mace,113.7,-28.5,1965-05-27,07_GrainFilling,2332.45,2233.923648,542.7,1965-09-01,...,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN
93,496,mace,113.7,-28.5,1966-05-27,07_GrainFilling,2301.45,2244.784630,311.5,1966-09-06,...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN


In [46]:
dfData['year'] = dfData['sowingdate'].dt.year
dfData['sowdate'] = dfData['sowingdate'].dt.strftime("%d-%B")
dfData

/apps/python/3.6.1/lib/python3.6/site-packages/IPython/kernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/apps/python/3.6.1/lib/python3.6/site-packages/IPython/kernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


,SimID,variety,long,lat,sowingdate,phases,season_cumAvgTemp,season_cumApsimTT,season_cumRain,startDate,...,days<=-2,days<=-3,days>=30,days>=32,days>=34,days>=36,days>=38,days>=40,year,sowdate
6,496,mace,113.7,-28.5,1957-05-27,07_GrainFilling,2318.05,2256.071910,409.9,1957-09-08,...,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,1957,27-May
15,496,mace,113.7,-28.5,1958-05-27,07_GrainFilling,2356.65,2279.326048,474.0,1958-09-04,...,NaN,NaN,2.0,2.0,NaN,NaN,NaN,NaN,1958,27-May
25,496,mace,113.7,-28.5,1959-05-27,07_GrainFilling,2208.85,2128.060868,362.2,1959-08-23,...,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,1959,27-May
35,496,mace,113.7,-28.5,1960-05-27,07_GrainFilling,2318.15,2247.336306,356.3,1960-09-09,...,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,1960,27-May
44,496,mace,113.7,-28.5,1961-05-27,07_GrainFilling,2268.80,2170.998617,334.5,1961-09-04,...,NaN,NaN,5.0,2.0,1.0,NaN,NaN,NaN,1961,27-May
53,496,mace,113.7,-28.5,1962-05-27,07_GrainFilling,2331.70,2262.822160,363.2,1962-08-30,...,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,1962,27-May
63,496,mace,113.7,-28.5,1963-05-27,07_GrainFilling,2252.45,2108.388637,457.1,1963-08-31,...,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,1963,27-May
73,496,mace,113.7,-28.5,1964-05-27,07_GrainFilling,2339.70,2237.335945,516.3,1964-08-26,...,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,1964,27-May
83,496,mace,113.7,-28.5,1965-05-27,07_GrainFilling,2332.45,2233.923648,542.7,1965-09-01,...,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,1965,27-May
93,496,mace,113.7,-28.5,1966-05-27,07_GrainFilling,2301.45,2244.784630,311.5,1966-09-06,...,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,1966,27-May


In [47]:
dfData = dfData[(dfData['year'] == filter_year)]
dfData

,SimID,variety,long,lat,sowingdate,phases,season_cumAvgTemp,season_cumApsimTT,season_cumRain,startDate,...,days<=-2,days<=-3,days>=30,days>=32,days>=34,days>=36,days>=38,days>=40,year,sowdate
6,496,mace,113.7,-28.5,1957-05-27,07_GrainFilling,2318.05,2256.07191,409.9,1957-09-08,...,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,1957,27-May


In [48]:
cols = dfData.columns.values.tolist()
print(cols)


['SimID', 'variety', 'long', 'lat', 'sowingdate', 'phases', 'season_cumAvgTemp', 'season_cumApsimTT', 'season_cumRain', 'startDate', 'endDate', 'dayCount', 'minTemp', 'maxTemp', 'avgTemp', 'ApsimTT', 'avgDaylength', 'cumAvgTemp', 'cumApsimTT', 'cumRain', 'avgWSDR', 'cumETpt', 'cumPARIO', 'avgPARIO', 'avgFDR', 'avgPQ', 'Biomass', 'LAI', 'Yield', 'days<=0', 'days<=-1', 'days<=-2', 'days<=-3', 'days>=30', 'days>=32', 'days>=34', 'days>=36', 'days>=38', 'days>=40', 'year', 'sowdate']


In [49]:
cols = ['SimID', 'variety', 'long', 'lat', 'sowdate', 'phases', 'dayCount', 
        'maxTemp', 'avgTemp', 'days>=30', 'days>=32']
dfData = dfData[cols]
dfData

,SimID,variety,long,lat,sowdate,phases,dayCount,maxTemp,avgTemp,days>=30,days>=32
6,496,mace,113.7,-28.5,27-May,07_GrainFilling,42,23.033333,17.145238,2.0,NaN


In [50]:
dfData

,SimID,variety,long,lat,sowdate,phases,dayCount,maxTemp,avgTemp,days>=30,days>=32
6,496,mace,113.7,-28.5,27-May,07_GrainFilling,42,23.033333,17.145238,2.0,NaN


In [51]:
dfData.rename(columns={'days>=30': 'daysGreaterEqual30', 'days>=32': 'daysGreaterEqual32'}, inplace=True)
dfData

/apps/python/3.6.1/lib/python3.6/site-packages/pandas/core/frame.py:3027: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


,SimID,variety,long,lat,sowdate,phases,dayCount,maxTemp,avgTemp,daysGreaterEqual30,daysGreaterEqual32
6,496,mace,113.7,-28.5,27-May,07_GrainFilling,42,23.033333,17.145238,2.0,NaN


In [52]:
outfile = sourcedir + "/" + filter_phase + "_" + str(filter_year) + ".csv"
outfile


'/OSM/CBR/AG_WHEATTEMP/work/output/07_GrainFilling_1957.csv'

In [53]:
if not os.path.isfile(outfile):
    dfData.to_csv(outfile, header=True, encoding='utf-8', index=False)
else:
    dfData.to_csv(outfile, header=False, mode='a', encoding='utf-8', index=False)
    

In [64]:
outfilelist = sourcedir + "/filelist.txt"
outfilelist


'/OSM/CBR/AG_WHEATTEMP/work/output/filelist.txt'

In [65]:
filelist_df.to_csv(outfilelist, header=False, index=False)

In [74]:
filelist_df = pd.read_csv(outfilelist, header=None)
filelist_df.columns=['filename']
filelist_df.head()

,filename
0,/OSM/CBR/AG_WHEATTEMP/work/output/113.60_-28.3...
1,/OSM/CBR/AG_WHEATTEMP/work/output/113.70_-28.4...
2,/OSM/CBR/AG_WHEATTEMP/work/output/113.70_-28.5...
3,/OSM/CBR/AG_WHEATTEMP/work/output/113.75_-28.4...
4,/OSM/CBR/AG_WHEATTEMP/work/output/113.75_-28.5...
